## Retrieve and clean data from website using selenium and pandas.
### Go to an e-commerce website, search for a product, ask how many pages need to visit, check all the details, make a data frame using PD, and generate an excel sheet.

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd

In [3]:
PATH = 'C:\Program Files (x86)\chromedriver.exe'

In [4]:
page_number = int(input("How many pages do you want to go after first one?"))

driver = webdriver.Chrome(PATH)
driver.get('https://evaly.com.bd/')
# driver.maximize_window()

try:
    ad_close = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH,"//button[@class='absolute top-0 right-0 p-2 text-black']"))
    ) # sometimes the window pops up. sometimes it doen't !!
    ad_close.click()
except:
    pass

try:
    search_product = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH,"//input[@aria-controls='listbox--1']"))
    )
    search_product.click()
    search_product.clear()                # clear the input box first
    search_product.send_keys("Samsung") 
    search_product.send_keys(Keys.RETURN) # hit enter
    
    # Make empty list, later append these texts from the element.
    name_list = []
    store_list = []
    price_list = []
    
    def page_elements():
        name_of_products = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH,"//div[@class='p-2']//p[@class='text-sm font-semibold leading-7 text-center text-gray-800']"))
        ) # as this is a list. need to use: presence_of_all_elements_located
        
        name_of_stores = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH,"//div[@class='p-2']//p[@class='text-sm font-semibold text-center text-gray-600']"))
        )
        
        product_price = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH,"//div[@class='p-2']//p[@class='SearchProductCard___StyledP2-sc-1cl8oip-1 gRXyNA']|//p[@class='SearchProductCard___StyledP3-sc-1cl8oip-2 hdejTW']"))
        )
        
        for i in name_of_products: # itterate through the list and appending item into the new list
            name_list.append(i.text)

        for i in name_of_stores:
            store_list.append(i.text)

        for i in product_price:
            price_list.append(i.text)
    

    page_elements() # always print the first page from landing page. for next page need to work more
    
    for i in range((page_number - 1)): # first page always print. so, range will be: given page num - number of page already printed 
        next_page = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH,"//ul[@class='pagination']/li/a[@aria-label='Page {}']".format(i+2)))
        ) # clicking next btn wont work. so, going one by one page. need to start clicking from page 2 to till last page number
        next_page.click()
        time.sleep(2)
        page_elements()
            
    print('Total page visited: {}'.format(page_number), ', and total product found: ', len(name_list))
except: # if the text did not find then, pass it
    print('Sorry, there is a problem')

Total page visited: 4 , and total product found:  80


**Make a dictionary so that we can convert it into DataFrame.** We also can take item.text from the elements list and store in a dictionary, using a for loop. Then, put the dict in a list. This list can directly be used in pandas dataframe.

In [5]:
product_dict = {'Product Name': name_list,
                'Store': store_list,
                'Price': price_list}

**Making DataFrame with pandas** 

In [6]:
df = pd.DataFrame.from_dict(product_dict)

**Data cleaning as '৳' not looking cool.**

In [7]:
df['Price'] = df['Price'].apply(lambda z: z.replace('৳', 'TK: '))
df

,Product Name,Store,Price
0,Samsung RT34K5532S8/D3 Refrigerator - 321L,Samsung Electronics for Priority Store,TK: 48300
1,Samsung 32'' UA32N4010ARSFS LED TV - Black,Samsung Electronics for Priority Store,TK: 25462
2,Samsung AX90R7080WD/EU Air Purifier - White,Samsung Electronics for Priority Store,TK: 41444
3,Samsung AX40R3030WM/EU Air Purifier - White,Samsung Electronics for Priority Store,TK: 22409
4,Samsung AR18TVHYDWKUFE Air Conditioner-1.5 Ton...,Samsung Electronics for Priority Store,TK: 79022
...,...,...,...
75,Samsung Micro SD Memory Card - 32GB - Black,Smart Gadgets Bangladesh for Cash on Delivery ...,TK: 499
76,Samsung Super Fast PD Adapter - 25W - Black,Nextera Design & Sourcing for Cash on Delivery...,TK: 1500
77,Samsung Super Fast PD Adapter - 45W - Black,Nextera Design & Sourcing for Cash on Delivery...,TK: 1925
78,Samsung Micro SD Memory Card - 64 GB - Black -...,GADGET KHOR for Cash on Delivery Service,TK: 980


**Generating excel file**

In [8]:
df.to_excel('product details.xlsx', index = False)